In [ ]:
from speechsep.imports import *
from speechsep.data import *
from speechsep.core import *

In [ ]:
class SiameseSpec(Tuple):
    def show(self, ctx=None, **kwargs):
        self[0].show()
        self[1].show()

In [ ]:
class SiamesePiar(Transform):
    def __init__(self, items, labels):
        store_attr(self, 'items,labels')
        self.assoc = self
        sortlbl = sorted(enumerate(labels), key=itemgetter(1))
        self.clsmap = {k:L(v).itemgot(0) for k,v in itertools.groupby(sortlbl, key=itemgetter(1))}
        self.idxs = range_of(self.items)
        
    def encodes(self,i)->None:
        othercls=self.clsmap[self.labels[i]] if random.random()>0.5 else self.idxs
        otherit = random.choice(othercls)
        return SiameseSpec(self.items[i], self.items[otherit])

In [ ]:
items = get_audio_files("../data/SmallDataset/")

In [ ]:
sp = SiamesePiar(items, items.map(parent_label))
maskify = Transform(MaskBinary.create, as_item=True)
pipe = Pipeline([sp, AudioMono.create, Resample(22050), Clip(5)])

In [ ]:
maskify(pipe(0))[1][0].data.shape

AttributeError: type object 'MaskBinary' has no attribute 'spec_func'

In [ ]:
tuple((1,2))

In [ ]:
(1,2)